In [1]:
import pandas    as pd
import seaborn   as sea 
import matplotlib.pyplot as plt
from pyspark.sql       import SparkSession
from pyspark.sql       import functions as f
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder\
    .appName('ProjetoSprint01')\
    .getOrCreate()
spark.sparkContext

<SparkContext master=local[*] appName=ProjetoSprint01>

### Carregando dados usando pyspark
> Defini o tipo do Schema(com o `StructType`) para alguns Conj. dados pois estavam tipando errado o schema, por exemplo, **`geolocation_zip_code_prefix`** que seus atributos são do tipo string contedo só numero de caracteres, mas estavam sendo colocados como do tipo `integer`.

In [4]:
# olist_customers_dataset
customers     = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_customers_dataset.csv')

# olist_geolocation_dataset
schemaGeo = StructType([ 
    StructField("geolocation_zip_code_prefix", StringType(), True),
    StructField("geolocation_lat", DoubleType(), True),
    StructField("geolocation_lng", DoubleType(), True),
    StructField("geolocation_city", StringType(), True),
    StructField("geolocation_state", StringType(), True)
])
geolocation   = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').schema(schemaGeo)\
        .load('dataset/olist_geolocation_dataset.csv')

# olist_order_payments_dataset
payments      = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_payments_dataset.csv')

# olist_orders_dataset
orders        = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_orders_dataset.csv')

# olist_order_items_dataset
order_items   = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_items_dataset.csv')

# olist_order_reviews_dataset
order_reviews = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_order_reviews_dataset.csv')

# olist_products_dataset
order_product = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/olist_products_dataset.csv')

# olist_sellers_dataset
schemaSeller = StructType([
    StructField("seller_id", StringType(), True),
    StructField("seller_zip_code_prefix", StringType(), True),
    StructField("seller_city", StringType(), True),
    StructField("seller_state", StringType(), True)
])
sellers      = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').schema(schemaSeller)\
        .load('dataset/olist_sellers_dataset.csv')

# product_category_dataset
product_cat  = spark.read.format('csv')\
    .option('header','True').option('delimiter',',').option('inferschema','True')\
        .load('dataset/product_category_name_translation.csv')

Criando uma visão temporaria , pois usarei o sql do spark para fazer as questões e melhor de entender oq estou fazendo

In [5]:
customers.createOrReplaceTempView('customers')
geolocation.createOrReplaceTempView('geolocation')
orders.createOrReplaceTempView('orders')
order_items.createOrReplaceTempView('order_items')
order_product.createOrReplaceTempView('order_product')
order_reviews.createOrReplaceTempView('order_reviews')
payments.createOrReplaceTempView('payments')
sellers.createOrReplaceTempView('sellers')
product_cat.createOrReplaceTempView('product_cat')

## Analise

Verificação de dados Nulos, fazerei uma função que recebe como parametros string(`nome do conj. dados`) e data(`conj. de dados em si`) ele irá me printar quantidade de valores nulos por coluna 

In [6]:
def VerificaValoresNulos(string, data):
    selecao = data.select(
        [ f.count( f.when(f.isnull(col), col)).alias(col)  for col in data.columns ]
    ).toPandas().T

    selecao.columns = ['Quantidade de valores Nulos']
    print(f'Para o conjuto de dados {string.title()} , com {data.count()} linhas')
    display(selecao.head())

In [16]:
VerificaValoresNulos('customers',customers)

Para o conjuto de dados Customers , com 99441 linhas


,Quantidade de valores Nulos
customer_id,0
customer_unique_id,0
customer_zip_code_prefix,0
customer_city,0
customer_state,0


In [19]:
VerificaValoresNulos('orders',orders)
print('\n')
VerificaValoresNulos('order_items',order_items)
print('\n')
VerificaValoresNulos('order_product',order_product)

Para o conjuto de dados Orders , com 99441 linhas


,Quantidade de valores Nulos
order_id,0
customer_id,0
order_status,0
order_purchase_timestamp,0
order_approved_at,160




Para o conjuto de dados Order_Items , com 112650 linhas


,Quantidade de valores Nulos
order_id,0
order_item_id,0
product_id,0
seller_id,0
shipping_limit_date,0




Para o conjuto de dados Order_Product , com 32951 linhas


,Quantidade de valores Nulos
product_id,0
product_category_name,610
product_name_lenght,610
product_description_lenght,610
product_photos_qty,610


In [20]:
VerificaValoresNulos('order_reviews',order_reviews)

Para o conjuto de dados Order_Reviews , com 104162 linhas


,Quantidade de valores Nulos
review_id,1
order_id,2236
review_score,2380
review_comment_title,92157
review_comment_message,63079


In [21]:
VerificaValoresNulos('payments',payments)

Para o conjuto de dados Payments , com 103886 linhas


,Quantidade de valores Nulos
order_id,0
payment_sequential,0
payment_type,0
payment_installments,0
payment_value,0


In [23]:
VerificaValoresNulos('sellers',sellers)

Para o conjuto de dados Sellers , com 3095 linhas


,Quantidade de valores Nulos
seller_id,0
seller_zip_code_prefix,0
seller_city,0
seller_state,0
